In [ ]:
#!/usr/bin/env python3
"""
Stage 2.2: Train CNN Model for Vulnerability Classification
Trains a Convolutional Neural Network on generated images
to classify circuit vulnerability (Low/Medium/High)

Model: Custom CNN or ResNet18
Framework: PyTorch
"""

import os
import sys
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from typing import Dict, Tuple, List

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Project root
PROJECT_ROOT = Path(__file__).resolve().parent.parent
DATASET_DIR = PROJECT_ROOT / 'dataset'
IMAGES_DIR = DATASET_DIR / 'images' / 'all'
MODELS_DIR = PROJECT_ROOT / 'models'
RESULTS_DIR = PROJECT_ROOT / 'results'

# Create directories
MODELS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# ================== CONFIGURATION ==================

# Training hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 50
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4

# Image parameters
IMAGE_SIZE = 224
NUM_CLASSES = 3

# Class mapping
CLASS_NAMES = ['Low', 'Medium', 'High']
CLASS_TO_IDX = {'Low': 0, 'Medium': 1, 'High': 2}
IDX_TO_CLASS = {0: 'Low', 1: 'Medium', 2: 'High'}

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🚀 Using device: {DEVICE}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")

# ================== DATASET CLASS ==================

class VulnerabilityDataset(Dataset):
    """Custom Dataset for vulnerability images"""
    
    def __init__(self, dataframe: pd.DataFrame, images_dir: Path, transform=None):
        """
        Args:
            dataframe: DataFrame with columns ['filename', 'label', 'split']
            images_dir: Directory containing images
            transform: Optional transform to be applied on images
        """
        self.dataframe = dataframe.reset_index(drop=True)
        self.images_dir = images_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Get image info
        row = self.dataframe.iloc[idx]
        circuit_name = row['circuit_name']
        sample_id = row['sample_id']
        label_str = row['vulnerability_label']
        
        # Construct filename
        filename = f"{circuit_name}_{sample_id:05d}_{label_str}.png"
        img_path = self.images_dir / filename
        
        # Load image
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            # Return a blank image if error
            image = Image.new('RGB', (IMAGE_SIZE, IMAGE_SIZE), color=(0, 0, 0))
        
        # Get label
        label = CLASS_TO_IDX[label_str]
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        return image, label

# ================== DATA TRANSFORMS ==================

def get_transforms(augment: bool = False):
    """
    Get image transforms
    
    Args:
        augment: Whether to apply data augmentation
    """
    if augment:
        # Training transforms with augmentation
        transform = transforms.Compose([
            transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
    else:
        # Validation/Test transforms (no augmentation)
        transform = transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
    
    return transform

# ================== CNN MODEL ARCHITECTURES ==================

class CustomCNN(nn.Module):
    """Custom CNN architecture for vulnerability classification"""
    
    def __init__(self, num_classes=3):
        super(CustomCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # 224 -> 112
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)  # 112 -> 56
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)  # 56 -> 28
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)  # 28 -> 14
        
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool5 = nn.MaxPool2d(2, 2)  # 14 -> 7
        
        # Fully connected layers
        self.fc1 = nn.Linear(512 * 7 * 7, 1024)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 256)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256, num_classes)
        
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        # Conv block 1
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        
        # Conv block 2
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        
        # Conv block 3
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        
        # Conv block 4
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)
        
        # Conv block 5
        x = self.relu(self.bn5(self.conv5(x)))
        x = self.pool5(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        
        return x

def get_resnet18_model(num_classes=3, pretrained=True):
    """
    Get ResNet18 model (transfer learning)
    
    Args:
        num_classes: Number of output classes
        pretrained: Whether to use pretrained weights
    """
    model = models.resnet18(pretrained=pretrained)
    
    # Modify final layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    
    return model

# ================== TRAINING FUNCTIONS ==================

class Trainer:
    """Trainer class for CNN model"""
    
    def __init__(self, model, device, model_name='CustomCNN'):
        self.model = model.to(device)
        self.device = device
        self.model_name = model_name
        
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=5, verbose=True
        )
        
        self.train_losses = []
        self.val_losses = []
        self.train_accs = []
        self.val_accs = []
        
        self.best_val_acc = 0.0
        self.best_model_path = None
    
    def train_epoch(self, dataloader):
        """Train for one epoch"""
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        pbar = tqdm(dataloader, desc='Training')
        for images, labels in pbar:
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            # Forward pass
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)
            
            # Backward pass
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            # Statistics
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Update progress bar
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
        
        epoch_loss = running_loss / total
        epoch_acc = 100. * correct / total
        
        return epoch_loss, epoch_acc
    
    def validate(self, dataloader):
        """Validate the model"""
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            pbar = tqdm(dataloader, desc='Validation')
            for images, labels in pbar:
                images = images.to(self.device)
                labels = labels.to(self.device)
                
                # Forward pass
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                # Statistics
                running_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                
                # Update progress bar
                pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{100.*correct/total:.2f}%'
                })
        
        epoch_loss = running_loss / total
        epoch_acc = 100. * correct / total
        
        return epoch_loss, epoch_acc
    
    def train(self, train_loader, val_loader, num_epochs):
        """Train the model"""
        print(f"\n{'='*70}")
        print(f"TRAINING {self.model_name}")
        print(f"{'='*70}")
        print(f"Device: {self.device}")
        print(f"Epochs: {num_epochs}")
        print(f"Batch Size: {BATCH_SIZE}")
        print(f"Learning Rate: {LEARNING_RATE}")
        print(f"{'='*70}\n")
        
        start_time = time.time()
        
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            print("-" * 70)
            
            # Train
            train_loss, train_acc = self.train_epoch(train_loader)
            self.train_losses.append(train_loss)
            self.train_accs.append(train_acc)
            
            # Validate
            val_loss, val_acc = self.validate(val_loader)
            self.val_losses.append(val_loss)
            self.val_accs.append(val_acc)
            
            # Learning rate scheduling
            self.scheduler.step(val_loss)
            
            # Print epoch results
            print(f"\nEpoch {epoch+1} Results:")
            print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"  Val Loss:   {val_loss:.4f}, Val Acc:   {val_acc:.2f}%")
            
            # Save best model
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                self.best_model_path = MODELS_DIR / f'{self.model_name}_best.pth'
                torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'val_acc': val_acc,
                    'val_loss': val_loss,
                }, self.best_model_path)
                print(f"  ✓ Saved best model (val_acc: {val_acc:.2f}%)")
        
        training_time = time.time() - start_time
        print(f"\n{'='*70}")
        print(f"TRAINING COMPLETED")
        print(f"{'='*70}")
        print(f"Total Training Time: {training_time/60:.2f} minutes")
        print(f"Best Validation Accuracy: {self.best_val_acc:.2f}%")
        print(f"Best Model Saved: {self.best_model_path}")
        print(f"{'='*70}\n")
    
    def plot_training_history(self):
        """Plot training history"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss plot
        ax1.plot(self.train_losses, label='Train Loss', marker='o')
        ax1.plot(self.val_losses, label='Val Loss', marker='s')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.set_title(f'{self.model_name} - Training and Validation Loss')
        ax1.legend()
        ax1.grid(True)
        
        # Accuracy plot
        ax2.plot(self.train_accs, label='Train Accuracy', marker='o')
        ax2.plot(self.val_accs, label='Val Accuracy', marker='s')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy (%)')
        ax2.set_title(f'{self.model_name} - Training and Validation Accuracy')
        ax2.legend()
        ax2.grid(True)
        
        plt.tight_layout()
        plot_path = RESULTS_DIR / f'{self.model_name}_training_history.png'
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved training history plot: {plot_path}")
        plt.close()

def evaluate_model(model, dataloader, device, class_names, model_name='Model'):
    """Evaluate model on test set"""
    print(f"\n{'='*70}")
    print(f"EVALUATING {model_name} ON TEST SET")
    print(f"{'='*70}")
    
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc='Testing'):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{model_name} - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    
    cm_path = RESULTS_DIR / f'{model_name}_confusion_matrix.png'
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')
    print(f"\n✓ Saved confusion matrix: {cm_path}")
    plt.close()
    
    # Calculate accuracy
    accuracy = 100. * (all_preds == all_labels).sum() / len(all_labels)
    print(f"\nTest Accuracy: {accuracy:.2f}%")
    print(f"{'='*70}\n")
    
    return accuracy, cm

# ================== MAIN EXECUTION ==================

def main():
    """Main execution function"""
    print("""
╔══════════════════════════════════════════════════════════════════════════╗
║                                                                          ║
║              STAGE 2.2: CNN MODEL TRAINING                               ║
║                                                                          ║
║    Hardware Trojan Vulnerability Assessment Project                      ║
║                                                                          ║
╚══════════════════════════════════════════════════════════════════════════╝
    """)
    
    # Load dataset
    print("\nLoading dataset...")
    df = pd.read_csv(DATASET_DIR / 'dataset_with_splits.csv')
    
    # Split data
    train_df = df[df['split'] == 'train']
    val_df = df[df['split'] == 'val']
    test_df = df[df['split'] == 'test']
    
    print(f"Train samples: {len(train_df)}")
    print(f"Val samples:   {len(val_df)}")
    print(f"Test samples:  {len(test_df)}")
    
    # Create datasets
    train_dataset = VulnerabilityDataset(train_df, IMAGES_DIR, transform=get_transforms(augment=True))
    val_dataset = VulnerabilityDataset(val_df, IMAGES_DIR, transform=get_transforms(augment=False))
    test_dataset = VulnerabilityDataset(test_df, IMAGES_DIR, transform=get_transforms(augment=False))
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # Option 1: Custom CNN
    print("\n" + "="*70)
    print("OPTION 1: TRAINING CUSTOM CNN")
    print("="*70)
    
    model_custom = CustomCNN(num_classes=NUM_CLASSES)
    trainer_custom = Trainer(model_custom, DEVICE, model_name='CustomCNN')
    trainer_custom.train(train_loader, val_loader, num_epochs=NUM_EPOCHS)
    trainer_custom.plot_training_history()
    
    # Load best model and evaluate
    checkpoint = torch.load(trainer_custom.best_model_path)
    model_custom.load_state_dict(checkpoint['model_state_dict'])
    acc_custom, cm_custom = evaluate_model(model_custom, test_loader, DEVICE, CLASS_NAMES, 'CustomCNN')
    
    # Option 2: ResNet18 (Transfer Learning)
    print("\n" + "="*70)
    print("OPTION 2: TRAINING RESNET18 (TRANSFER LEARNING)")
    print("="*70)
    
    model_resnet = get_resnet18_model(num_classes=NUM_CLASSES, pretrained=True)
    trainer_resnet = Trainer(model_resnet, DEVICE, model_name='ResNet18')
    trainer_resnet.train(train_loader, val_loader, num_epochs=NUM_EPOCHS)
    trainer_resnet.plot_training_history()
    
    # Load best model and evaluate
    checkpoint = torch.load(trainer_resnet.best_model_path)
    model_resnet.load_state_dict(checkpoint['model_state_dict'])
    acc_resnet, cm_resnet = evaluate_model(model_resnet, test_loader, DEVICE, CLASS_NAMES, 'ResNet18')
    
    # Summary
    print("""
╔══════════════════════════════════════════════════════════════════════════╗
║                                                                          ║
║            ✓ STAGE 2.2 CNN TRAINING COMPLETED                            ║
║                                                                          ║
╚══════════════════════════════════════════════════════════════════════════╝

MODEL COMPARISON:
────────────────────────────────────────────────────────────────────────
Custom CNN Test Accuracy:    {:.2f}%
ResNet18 Test Accuracy:      {:.2f}%

Best Model: {}

FILES SAVED:
────────────────────────────────────────────────────────────────────────
Models:
  ├─ CustomCNN_best.pth
  └─ ResNet18_best.pth

Results:
  ├─ CustomCNN_training_history.png
  ├─ CustomCNN_confusion_matrix.png
  ├─ ResNet18_training_history.png
  └─ ResNet18_confusion_matrix.png

Location: {}
────────────────────────────────────────────────────────────────────────
    """.format(
        acc_custom, 
        acc_resnet,
        "ResNet18" if acc_resnet > acc_custom else "Custom CNN",
        RESULTS_DIR
    ))

if __name__ == "__main__":
    main()
